In [1]:
import math
import warnings
import numpy as np
import pandas as pd

from tqdm import tqdm
from matplotlib import pyplot as plt


warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
# Retrieve the FBK dataset and set Time column as the new index
appa1 = pd.read_csv("data/appa1.csv")
appa1 = appa1.drop(columns='Unnamed: 0')
appa1 = appa1.set_index("Time")

# Retrieve the APPA dataset (our ground truth)
appa1_gt = pd.read_csv("appa1_gt.csv")

: 

: 

# Our data

| **Original column name** |       **New column name**      |                                    **Description**                                    |
|:------------------------:|:------------------------------:|:-------------------------------------------------------------------------------------:|
|            ts            |              Time              |                                    Time of sampling                                   |
|          Rs<1-8>         |    <sensing_material>_<1/2>    |                   Resistance of the corresponding sensor                   |
|          Rh<1-8>         | <sensing_material>_<1/2>_heatR |                   Resistance of the corresponding heater                   |
|          Vh<1-8>         | <sensing_material>_<1/2>_heatV |                                 Voltage of the corresponding heater                                 |
|             T            |           Temperature          |                                      Temperature                                      |
|            RH            |        Relative_Humidity       |                                   Relative humidity                                   |
|             P            |            Pressure            |                                      Air pressure                                     |
|            BME           |               VOC              | VOC (Volatile Organic Compounds) extracted from BME (data of a commercial gas sensor) |
|             -            |  <sensing_material>_<1/2>_Age  |               The difference between current day and day of installation              |
|             -            |       sin_hour, cos_hour       |                                    Periodical hour                                    |
|             -            |    sin_weekday, cos_weekday    |                               Periodical day of the week                              |
|             -            |      sin_month, cos_month      |                                    Periodical month                                   |
|             -            |     sin_ordate, cos_ordate     |                       Periodical day of the year (ordinal date)                       |
|             -            |              year              |                                          Year                                         |
|     Biossido di Azoto    |               NO2              |                             Nitrogen dioxide concentration                            |
|      Biossido Zolfo      |               SO2              |                              Sulfur dioxide concentration                             |
|           Ozono          |               O3               |                                  Ozone concentration                                  |
|    Ossido di Carbonio    |               CO               |                             Carbon monoxide concentration                             |

In [3]:
# ???
appa1.at[381849, "Time"] = pd.to_datetime("Tue Jan 12 2021 15:39:26 GMT+0000")
+ pd.to_timedelta(1, unit="min")

# Fix column names to match convention
appa1 = appa1.rename(columns={
    "ts": "Time",
    "T": "Temperature",
    "RH": "Relative_Humidity",
    "P": "Pressure",
    "BME": "VOC"
    })

In [10]:
# Interpolate NaN values
for i, row in tqdm(appa1.iterrows(), total=len(appa1)):
    if type(row.Time) != str:
        print(i, appa1.iloc[i-1].Time, appa1.iloc[i].Time)
        appa1.at[i, "Time"] = pd.to_datetime(" ".join(appa1.iloc[i-1].Time.split(" ")[:6])) + pd.to_timedelta(1, unit="m")

  0%|          | 0/1183842 [00:00<?, ?it/s]

: 

: 

In [ ]:
# Convert Timestamps to Datetime format
appa1.Time = pd.to_datetime(
    appa1.Time.map(lambda x:  " ".join(x.split(" ")[:6]) if type(x) == str else x),
    utc=True)

appa1_gt.Time = pd.to_datetime(appa1_gt.Time)

In [7]:
# ???
for i in tqdm(range(0, len(appa1)-60, 60), total=int(len(appa1)-60)/60):
       row = appa1.iloc[i:i+60].mean(axis=0)
       temp = [appa1.iloc[i].Time]
       temp[1:] = row
       appa1.loc[len(appa1.index)] = temp

  0%|          | 0/19729.7 [00:00<?, ?it/s]


AttributeError: 'Series' object has no attribute 'Time'

In [ ]:
# ???
for i, row in tqdm(appa1.iterrows(), total = len(appa1)):
    appa1.at[i, "time"] = row.Time.replace(microsecond=0, second=0, minute=0)

# appa1_hr.Time = appa1_hr.Time.dt.round('60min')

100%|██████████| 19730/19730 [00:00<00:00, 25288.23it/s]


In [9]:
# ???
appa1.Time = pd.to_datetime(appa1.Time)
appa1.Time = appa1.Time.dt.tz_localize(None)

AttributeError: 'DataFrame' object has no attribute 'Time'

In [ ]:
# Insert ground truth columns to the FBK dataset
appa1.merge(appa1_gt, how='left', on="Time")
appa1.iloc[380]

In [ ]:
#6380
#3937
#17641

appa1 = appa1.iloc[6380:]
print(len(appa1))

13350


In [ ]:
# ???
appa1_gt = appa1_gt.iloc[3864:17569]
print(len(appa1_gt))

13705


In [ ]:
appa1_clean = appa1[appa1.Time >= '2021-01-14']
appa1_first = appa1_clean[appa1_clean.Time <= '2021-04-29']
appa1_second = appa1_clean[(appa1_clean.Time > '2021-04-29') & (appa1_clean.Time <= '2021-6-30')]
appa1_third = appa1_clean[(appa1_clean.Time > '2021-06-30') & (appa1_clean.Time <= '2022-1-14')]
appa1_fourth = appa1_clean[appa1_clean.Time > '2022-1-14']

In [ ]:
appa1_second = appa1_second.iloc[:, [0,
 1,
 2,
 5, #
 6, #
 3, #
 4, #
 7,
 8,

 9,
 10,
 13, #
 14, #
 11, #
 12, #
 15,
 16,

 17,
 18,
 21, #
 22, #
 19, #
 20, #
 23,
 24,
 25,
 26,
 27,
 28]]

appa1_third = appa1_third.iloc[:, [0,
5,6,1,2,3,4,7,8,

13,14,9,10,11,12,15,16,

 21,22,17,18,19,20,23,24,

 25,
 26,
 27,
 28]]

appa1_fourth = appa1_fourth.iloc[:, [0,
5,6,1,2,3,4,7,8,

13,14,9,10,11,12,15,16,

 21,22,17,18,19,20,23,24,

 25,
 26,
 27,
 28]]

In [ ]:
appa1 = pd.DataFrame(
       np.vstack([appa1_first, appa1_second, appa1_third, appa1_fourth]),
       columns=[
              "Time", 

              "ZnOR_1",
              "ZnOR_2",
              "LaFeO3_1",
              "LaFeO3_2",
              "WO3_1",
              "WO3_2", 
              "to_del", 
              "to_del2",

              "ZnOR_heatR_1",
              "ZnOR_heatR_2",
              "LaFeO3_heatR_1",
              "LaFeO3_heatR_2",
              "WO3_heatR_1",
              "WO3_heatR_2", 
              "to_de3", 
              "to_del1",

              "ZnOR_heatV_1",
              "ZnOR_heatV_2",
              "LaFeO3_heatV_1",
              "LaFeO3_heatV_2",
              "WO3_heatV_1",
              "WO3_heatV_2", 
              "to_deal", 
              "to_de2",

              'Temperature', 
              'Relative_Humidity',
              'Pressure', 
              'VOC'
       ]
       )

In [ ]:
appa1 = appa1[[
              "Time", 

              "ZnOR_1",
              "ZnOR_2",
              "LaFeO3_1",
              "LaFeO3_2",
              "WO3_1",
              "WO3_2", 

              "ZnOR_heatR_1",
              "ZnOR_heatR_2",
              "LaFeO3_heatR_1",
              "LaFeO3_heatR_2",
              "WO3_heatR_1",
              "WO3_heatR_2", 

              "ZnOR_heatV_1",
              "ZnOR_heatV_2",
              "LaFeO3_heatV_1",
              "LaFeO3_heatV_2",
              "WO3_heatV_1",
              "WO3_heatV_2", 

              'Temperature', 
              'Relative_Humidity',
              'Pressure', 
              'VOC'
       ]]

In [ ]:
appa1 = appa1.set_index("Time")

In [ ]:
appa1["ZnOR_1_Age"] = 0
appa1["ZnOR_2_Age"] = 0
appa1["LaFeO3_1_Age"] = 0
appa1["LaFeO3_2_Age"] = 0
appa1["WO3_1_Age"] = 0
appa1["WO3_2_Age"] = 0

In [ ]:
appa1.index = pd.to_datetime(appa1.index)

In [ ]:
def norm(data):
    return (data)/(max(data))

appa1.loc[(appa1.index < "2021-06-30"), "ZnOR_2"] = norm(
    appa1[(appa1.index < "2021-06-30")].ZnOR_1) * max(
        appa1[(appa1.index > "2021-06-30")].ZnOR_2)

In [ ]:
appa1.Time = pd.to_datetime(appa1.Time)
appa1 = appa1.set_index("Time")

In [ ]:
"""
appa1_clean = appa1[appa1.Time >= '2021-01-14']
appa1_first = appa1_clean[appa1_clean.Time <= '2021-04-29']
appa1_second = appa1_clean[(appa1_clean.Time > '2021-04-29') & (appa1_clean.Time <= '2021-6-30')]
appa1_third = appa1_clean[(appa1_clean.Time > '2021-06-30') & (appa1_clean.Time <= '2022-1-14')]
appa1_fourth = appa1_clean[appa1_clean.Time > '2022-1-14']


appa1["ZnOR_1_Age"] = 0
appa1["ZnOR_2_Age"] = 0
appa1["LaFeO3_1_Age"] = 0
appa1["LaFeO3_2_Age"] = 0
appa1["WO3_1_Age"] = 0
appa1["WO3_2_Age"] = 0
"""

for i,row in appa1.iterrows():
    age = i.timestamp() - pd.Timestamp("2021-01-14").timestamp()

    appa1.loc[i,"ZnOR_1_Age"] = age
    appa1.loc[i,"ZnOR_2_Age"] = age
    appa1.loc[i,"LaFeO3_1_Age"] = age
    appa1.loc[i,"LaFeO3_2_Age"] = age
    appa1.loc[i,"WO3_1_Age"] = age
    appa1.loc[i,"WO3_2_Age"] = age
for i,row in appa1[appa1.index > "2021-4-29"].iterrows():
    age = i.timestamp() - pd.Timestamp("2021-4-29").timestamp()

    appa1.loc[i, "WO3_1_Age"] = age
    appa1.loc[i, "WO3_2_Age"] = age

In [ ]:
# Add periodical time features to the dataset

# HOUR OF THE DAY
appa1["sin_hour"] = appa1["Time"].dt.hour.apply(
    lambda x: math.sin(x) * 2 * (math.pi / 24)
)
appa1["cos_hour"] = appa1["Time"].dt.hour.apply(
    lambda x: math.cos(x) * 2 * (math.pi / 24)
)

# DAY OF THE WEEK
appa1["sin_weekday"] = appa1["Time"].dt.weekday.apply(
    lambda x: math.sin(x) * 2 * (math.pi / 7)
)
appa1["cos_weekday"] = appa1["Time"].dt.weekday.apply(
    lambda x: math.cos(x) * 2 * (math.pi / 7)
)

# MONTH OF THE YEAR
appa1["sin_month"] = appa1["Time"].dt.month.apply(
    lambda x: math.sin(x) * 2 * (math.pi / 12)
)

appa1["cos_month"] = appa1["Time"].dt.month.apply(
    lambda x: math.cos(x) * 2 * (math.pi / 12)
)

# DAY OF THE YEAR (ORDINAL DATE)
appa1["sin_ordate"] = appa1["Time"].apply(
    lambda x: math.sin(x.toordinal()) * 2 * (math.pi / 366)
)
appa1["cos_ordate"] = appa1["Time"].apply(
    lambda x: math.cos(x.toordinal()) * 2 * (math.pi / 366)
)

# Add the year feature
appa1["year"] = appa1["Time"].dt.year

In [ ]:
# Export as new clean dataset that will be used from now on
appa1.to_csv("exports/appa_hrgroup_time.csv")